# Laboratório 2: Construindo Sistemas com IA Generativa

## Parte 1 - RAG de Documentos

### 1. Interação LLM

In [1]:
import os
from langchain_openai import ChatOpenAI
from IPython.display import Markdown
from dotenv import load_dotenv

# Carregando variáveis de ambiente
load_dotenv()

True

In [39]:
# Criando interface para interagir com o modelo
model = ChatOpenAI(
    model="gpt-4o-mini",   # modelo leve e rápido
    temperature=0        # controla a criatividade da resposta
)

# Fazendo uma pergunta simples ao modelo
resposta = model.invoke(
    "Quais foram as principais dificuldades enfrentadas pelas FADM nas operações de guerrilha no Teatro Operacional Norte?"
    )
display(Markdown(resposta.content))

As Forças Armadas de Defesa de Moçambique (FADM) enfrentaram diversas dificuldades nas operações de guerrilha no Teatro Operacional Norte, especialmente no contexto do combate ao terrorismo e à insurgência em Cabo Delgado. Algumas das principais dificuldades incluem:

1. **Falta de Recursos**: As FADM frequentemente enfrentaram limitações em termos de equipamentos, armamentos e suprimentos, o que dificultou a eficácia das operações.

2. **Treinamento e Capacitação**: A falta de treinamento adequado e especializado para lidar com táticas de guerrilha e combate em terrenos difíceis comprometeu a capacidade das forças de responder de forma eficaz.

3. **Inteligência e Informação**: A dificuldade em obter informações precisas sobre os movimentos e a localização dos insurgentes prejudicou as operações, tornando-as menos eficazes.

4. **Terreno Desfavorável**: O terreno montanhoso e a densa vegetação de Cabo Delgado apresentaram desafios significativos para as operações militares, dificultando a mobilidade e a logística.

5. **População Civil**: A presença de civis na área de conflito complicou as operações, uma vez que as FADM precisavam evitar causar danos colaterais e ganhar a confiança da população local.

6. **Desmotivação e Moral**: A desmotivação entre as tropas, devido a condições adversas e à falta de apoio, pode ter impactado a moral e a eficácia das operações.

7. **Apoio Internacional**: A dependência de apoio externo, tanto em termos de treinamento quanto de recursos, pode ter limitado a autonomia das FADM nas operações.

Essas dificuldades, entre outras, contribuíram para a complexidade do conflito em Cabo Delgado e para os desafios enfrentados pelas FADM nas suas operações de combate à insurgência.

### 2. Interação LLM pura x RAG

### 3. Quebra do documento em chunks

Vamos quebrar o documentos em partes menores *(chunks)* que possuem alguma sobreposição entre si. A ideia é selecionar alguns desses *chunks* de acordo com a pergunta do usuário para serem usados como base contextual para a LLM responder a pergunta

In [4]:
from langchain.document_loaders import PyPDFLoader

pdf_path = "Relato-do-I-Simposio-sobre-Doutrina-Militar.pdf"

loader = PyPDFLoader(pdf_path)
docs = loader.load()

print(f"✅ Documento carregado com {len(docs)} páginas")
print(docs[1].page_content[:500])  # mostra os primeiros 500 caracteres da primeira página


✅ Documento carregado com 84 páginas
Academia Militar Marechal Samora Machel 
 
 
 
 
 
 
 
 
RELATO DO  
I SIMPÓSIO SOBRE DOUTRINA MILITAR 
 
 
 
 
 
 
 
 
 
Nampula, Setembro de 2022


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,      # número de caracteres por chunk
    chunk_overlap=150,   # sobreposição entre chunks
    separators=["\n\n", "\n", ".", " "]  # ordem de prioridade na divisão
)

# Aplicar o splitter ao documento
chunks = text_splitter.split_documents(docs)

print(f"✅ Documento dividido em {len(chunks)} chunks")


✅ Documento dividido em 206 chunks


In [6]:
for i, chunk in enumerate(chunks[20:30]):
    print(f"\n--- Chunk {i+1} ---")
    print(chunk.page_content)  # mostrar apenas os primeiros 300 caracteres


--- Chunk 1 ---
4. Doutrinas de outros países tendo em atenção os aspectos culturais do nosso país. 
Com efeito, a Doutrina M ilitar deve definir as linhas de comando (estratégico, operacional e 
táctico) e ref lectir co m ideias e entendimentos ao nível do s Ramos e ao nível combinado 
(operações militares que envolvem duas ou mais Nações), apresentados em manuais de 
campanha, cadernos, notas e materiais de instrução, para estabelecer uma forma de ser e estar 
das FADM. 
Nessa matriz doutrinária, as instituições de ensino militar devem estar na dianteira nas 
actividades de reflexão doutrinária em todos os níveis (estratégico, operacional e táctico), 
assim como considerar o pressuposto de que as FADM não constituem local de absorção de

--- Chunk 2 ---
assim como considerar o pressuposto de que as FADM não constituem local de absorção de 
desempregados da sociedade e não apostam na quantidade, mas sim na qualidade dos homens. 
No entanto , para dar continuidade ao processo de refle

### 3. Vetorização de um chunk
Vamos usar um modelo de embedding para transformar o *chunk* em um vetor de 1536 dimensões que armazena um valor semântico daquele texto

In [7]:
from langchain_openai import OpenAIEmbeddings
import numpy as np

# Inicializa o modelo de embeddings da OpenAI
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Seleciona um dos chunks gerados anteriormente
exemplo_chunk = chunks[100].page_content
print("📄 Texto do chunk:\n", exemplo_chunk[:300])


📄 Texto do chunk:
 41 
DOUTRINA MILITAR DAS OPERAÇÕES NAVAIS DAS FADM 
Capitão-de-Mar-e-Guerra Carlos Miguel Castanheira Cossa* 
 
1. Introdução 
A Marinha de Guerra de Moçambique (MGM), um dos ramos das Forças Armadas de Defesa 
de Moçambique (FADM), é uma das instituições com elevada responsabilidade no âmbito da 
d


In [8]:
# Gera o vetor de embedding para o chunk selecionado
vetor = embedding_model.embed_query(exemplo_chunk)

print(f"\n✅ Embedding gerado! O vetor tem {len(vetor)} dimensões.")
print("🔢 Primeiros 10 valores do vetor:\n", vetor[:10])



✅ Embedding gerado! O vetor tem 1536 dimensões.
🔢 Primeiros 10 valores do vetor:
 [0.018671372905373573, 0.09511669725179672, 0.04760127142071724, 0.009668337181210518, -0.0038979174569249153, 0.031891562044620514, -0.011481821537017822, 0.010419484227895737, -0.02307095006108284, 0.009067419916391373]


### 4. Banco vetorial (Chroma)
- Vamos criar **um embedding para cada chunk** e armazena-lo em um banco vetorial (Chroma). 

- Cada chunk será armazenado no banco como um **vetor + texto do trecho + metadado**
    - Obs: Esse metadado pode ser o nome do documento de origem, o nº da página do trecho, etc

- A ideia do banco é que poderemos fazer **buscas semânticas** utilzando **cálculos vetoriais** para selecionar os trechos do documento que estão **mais relacionados** com uma determinada pergunta do usuário

In [9]:
from langchain_chroma import Chroma

vectorstore = Chroma(
    collection_name="doutrina-militar",
    embedding_function=embedding_model,
    persist_directory="./chroma_db" # Vamos persistir nossos dados em um banco sqlite na pasta chroma_db
)

In [10]:
# Adiciona os documentos chunkados ao banco vetorial
vectorstore.add_documents(chunks)

print("✅ Todos os chunks foram vetorizados e armazenados no Chroma!")

✅ Todos os chunks foram vetorizados e armazenados no Chroma!


In [11]:
collection = vectorstore._client.get_collection("doutrina-militar")

print(f"📊 O banco vetorial contém {collection.count()} embeddings.")

📊 O banco vetorial contém 206 embeddings.


### 5. Busca semântica no Chroma
- Vamos fazer uma busca semântica dentro do banco vetorial. 

- Para isso, a pergunta do usuário *(query)* será vetorizada.

- Será feita uma busca de proximidade cossenoidal do vetor da pergunta com todos os outro vetores no banco.

In [13]:
query = "O que o documento fala sobre guerrilha?"
resultados = vectorstore.similarity_search(query, k=5)

print("🔎 Trechos mais relevantes encontrados:")
for i, r in enumerate(resultados):
    print(f"\n--- Resultado {i+1} ---")
    print(r.page_content[:300])

🔎 Trechos mais relevantes encontrados:

--- Resultado 1 ---
os não limitados civis armados. É uma técnica de combate  baseada numa luta efectuada por 
pequenos grupos de combatente s, composto geralmente por pessoas não pertencentes ao 
Exército Regular nacional, baseada no desrespeito pelas convenções internacionais. 
A guerrilha é, igualmente, a modalidade

--- Resultado 2 ---
Coronel G. Muiambo 
38 
Doutrina Militar versus Operações de Guerrilha 
As características de operações de guerrilha compreendem: i rregularidade do combate,  feito 
pelos guerrilheiros;  capacidade de mobilidade;  compromisso político ( interno e externo ); 
carácter (ligação íntima com as populaçõ

--- Resultado 3 ---
Coronel G. Muiambo 
37 
Doutrina Militar versus Operações de Guerrilha 
combate (Terrestres, Aéreos e Navais)  e armamento modernos, com meios de comunicação 
(fixos e móveis) eficientes e uma Logística (alimentação/ração de combate,  equipamentos 
diversos, combustíveis e lubrificantes) à al tu

### 6. Finalização do RAG

In [19]:
def rag_module(pergunta: str, num_chunks: int = 5):
    chunks = vectorstore.similarity_search(query, k=num_chunks)
    contexto = "\n\n".join([r.page_content for r in chunks])

    prompt = f"""
        1- Você é um assistente especialista em responder perguntas com base no contexto em que lhe fornecido entre ===
        2- Use APENAS as informações abaixo para responder à pergunta.

        ===
        Contexto:
        {contexto}
        ===

        Pergunta: {pergunta}

        Responda de forma clara, concisa e cite os trechos do contexto quando possível.
    """

    resposta_final = model.invoke(prompt)

    return resposta_final


In [40]:
pergunta = "Quais foram as principais dificuldades enfrentadas pelas FADM nas operações de guerrilha no Teatro Operacional Norte?"

resposta = rag_module(pergunta)


print("\n💡 Resposta final do RAG:\n")

# Printar resposta
display(Markdown(resposta.content))


💡 Resposta final do RAG:



As principais dificuldades enfrentadas pelas Forças Armadas de Moçambique (FADM) nas operações de guerrilha no Teatro Operacional Norte (TON) incluem:

1. **Organização e Comando**: As FADM têm enfrentado desafios na estrutura organizacional e na liderança durante as operações de guerrilha.

2. **Meios de Combate**: Há deficiências nos meios de combate, que incluem recursos terrestres, aéreos e navais.

3. **Logística**: As dificuldades logísticas têm sido um obstáculo significativo, afetando o transporte e a disponibilidade de equipamentos militares.

4. **Equipamentos Militares**: A falta de equipamentos adequados, como fardamento, coletes de proteção, capacetes, cartucheiras e botas, tem comprometido a eficácia das operações.

Essas dificuldades são mencionadas no trecho: "Desde o início da guerra de terrorismo, em Cabo Delgado, as FADM têm vindo a debater-se com várias dificuldades, desde a organização, comando, meios de combate (terrestres, aéreos e navais), deficiências logísticas, transporte, equipamentos militares (fardamento, coletes de proteção, capacetes, cartucheiras, botas), entre outros."

### 7. Propostas de melhoria

- **Refinar prompt:** Pedir para o LLM refinar a pergunta antes de fazer a busca vetorial, com o objetivo de obter chunks mais adequados

- **Estratégia de criação de chunks:** Usar outras estratégias para quebra do documento em chunks
    - Usar a estrutura do documento como referência (Ex: Em uma legislação quebrar um chunk por artigo)
    - Fazer quebra em chunks usando LLM para criar chunks que englobem totalmente uma ideia

- **Usar metadados dos chunks:** Os chunks possuem metadados associados. Geralmente inclui o nome da sua fonte de origem, autor, etc. Mas esse campo pode ser editado a critério do desenvolvedor.

- **Ferramentas de extração de texto:** O langchain possui funções que permitem você extrair o texto de outras tipos de fonte. Por exemplo:
    - **PyPDFLoader**: Extrai o texto de PDFs
    - **YoutubeAudioLoader**: Extrai a transcrição de vídeos do YouTube automaticamente
    - **WebBaseLoader**: Faz scraping de páginas, inclusive via sitemap
    - **WikipediaLoader**: Busca e extrai artigos diretamente da Wikipédia.


#### ATENÇÃO Adicionar um diagrama de como RAG funciona

## Parte 2 - Agentes

### 1. Execução de funções

In [62]:
resposta = model.invoke("Quantas letras R tem a palavra CORREDOR?")
resposta.content


'A palavra "CORREDOR" tem 2 letras R.'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, initialize_agent
from langchain.agents import AgentType

# 🧮 Função que conta quantas vezes uma letra aparece em uma palavra
def contar_letra(entrada: str) -> str:
    """
    Espera uma string no formato 'palavra letra'
    Exemplo: 'banana a'
    """
    try:
        palavra, letra = entrada.split()
        contagem = palavra.lower().count(letra.lower())
        return f"A letra '{letra}' aparece {contagem} vez(es) na palavra '{palavra}'."
    except Exception as e:
        return f"Erro: forneça os dados no formato 'palavra letra'. Exemplo: 'banana a'"

# 🔧 Cria a tool com descrição para o agente entender
tools_exemplo = [
    Tool(
        name="Contador de letras",
        func=contar_letra,
        description="Conta quantas vezes uma letra específica aparece em uma palavra. Entrada esperada: 'palavra letra'."
    )
]

# 🧠 Cria o agente
agente_exemplo = initialize_agent(
    tools_exemplo,
    model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [64]:
resposta = agente_exemplo.invoke({"input": "Quantas letras R tem a palavra CORREDOR?"})
print("\n🤖 Resposta:", resposta["output"])




> Entering new AgentExecutor chain...
I need to count how many times the letter 'R' appears in the word "CORREDOR". 
Action: Contador de letras
Action Input: "CORREDOR R"
Observation: A letra 'R' aparece 3 vez(es) na palavra 'CORREDOR'.
Thought:I now know the final answer
Final Answer: A letra 'R' aparece 3 vezes na palavra 'CORREDOR'.

> Finished chain.

🤖 Resposta: A letra 'R' aparece 3 vezes na palavra 'CORREDOR'.


### 2. Prompt especialista

### 3. Busca no banco de dados

In [65]:
# Banco simulado
usuarios = {
    "João": {"cargo": "Engenheiro de IA", "setor": "Pesquisa"},
    "Maria": {"cargo": "Analista de Dados", "setor": "Inteligência"},
}

# Função de consulta
def consultar_usuario(nome: str) -> str:
    dados = usuarios.get(nome)
    if not dados:
        return "Usuário não encontrado."
    return f"{nome} é {dados['cargo']} e trabalha no setor de {dados['setor']}."

# Tool de busca
tools_bd = [
    Tool(
        name="Consulta ao banco de dados de usuários",
        func=consultar_usuario,
        description="Fornece informações sobre um usuário registrado no banco interno."
    )
]

# Recria o agente
agente_bd = initialize_agent(
    tools_bd,
    model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Teste
resposta = agente_bd.invoke({"input": "Quem é João?"})
print("\n📊 Resposta:", resposta["output"])




> Entering new AgentExecutor chain...
I need to find information about the user named João in the internal database.  
Action: Consulta ao banco de dados de usuários  
Action Input: "João"  
Observation: João é Engenheiro de IA e trabalha no setor de Pesquisa.
Thought:I now know the final answer  
Final Answer: João é Engenheiro de IA e trabalha no setor de Pesquisa.

> Finished chain.

📊 Resposta: João é Engenheiro de IA e trabalha no setor de Pesquisa.


### 4. Agente metereologico

In [93]:
import requests

# ============================
# 🔹 Tool 1 — Busca coordenadas da cidade
# ============================
# def buscar_coordenadas(cidade: str) -> str:
#     """Retorna as coordenadas (latitude, longitude) da cidade informada."""
#     cidades = {
#         "Brasília": (-15.78, -47.93),
#         "Rio de Janeiro": (-22.90, -43.17),
#         "São Paulo": (-23.55, -46.63),
#         "Maputo": (-25.97, 32.58),
#         "Nampula": (-15.12, 39.27),
#         "Beira": (-19.84, 34.84)
#     }
#     if cidade not in cidades:
#         return "Cidade não encontrada. Tente Brasília, Rio de Janeiro, São Paulo, Maputo, Nampula ou Beira."
    
#     lat, lon = cidades[cidade]
#     return f"{cidade}: latitude={lat}, longitude={lon}"

def buscar_coordenadas(cidade: str) -> str:
    """Busca latitude e longitude da cidade usando a API Nominatim (OpenStreetMap)."""
    url = f"https://nominatim.openstreetmap.org/search?q={cidade}&format=json&limit=1"
    headers = {"User-Agent": "LangChain-Agent-Demo"}
    
    r = requests.get(url, headers=headers)
    if not r.ok or not r.json():
        return "Não consegui encontrar coordenadas para essa cidade."
    
    dados = r.json()[0]
    lat, lon = dados["lat"], dados["lon"]
    return f"{cidade}: latitude={lat}, longitude={lon}"

# ============================
# 🔹 Tool 2 — Consulta clima via API Open-Meteo
# ============================
def clima_por_coordenadas(entrada: str) -> str:
    """Recebe uma string no formato 'latitude=X, longitude=Y' e retorna o clima atual."""
    try:
        partes = entrada.replace(",", "").split()
        lat = float(partes[0])
        lon = float(partes[1])
    except Exception as err:
        return f"Formato inválido. {err}"

    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    r = requests.get(url)
    dados = r.json().get("current_weather", {})
    if not dados:
        return "Não foi possível obter dados climáticos."
    
    temperatura = dados["temperature"]
    vento = dados["windspeed"]
    return f"Temperatura: {temperatura}°C | Vento: {vento} km/h"

# ============================
# 🧰 Define as tools
# ============================
tools_web = [
    Tool(
        name="Buscar coordenadas da cidade",
        func=buscar_coordenadas,
        description="Retorna latitude e longitude de uma cidade."
    ),
    Tool(
        name="Consultar clima por coordenadas",
        func=clima_por_coordenadas,
        description="Dado latitude e longitude, retorna a temperatura e velocidade do vento."
    )
]


# Recria o agente
agente = initialize_agent(
    tools_web,
    model,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [94]:
resposta = agente.invoke({"input": "Qual é o clima em Seoul?"})
print("\n🌤️ Resultado:", resposta["output"])



> Entering new AgentExecutor chain...
Para responder à pergunta sobre o clima em Seoul, primeiro preciso obter as coordenadas da cidade (latitude e longitude). 

Action: Buscar coordenadas da cidade  
Action Input: "Seoul"  
Observation: Seoul: latitude=37.5666791, longitude=126.9782914
Thought:Agora que tenho as coordenadas de Seoul, posso consultar o clima usando essas informações.

Action: Consultar clima por coordenadas  
Action Input: "37.5666791, 126.9782914"  
Observation: Temperatura: 15.0°C | Vento: 5.9 km/h
Thought:Agora que tenho as informações sobre a temperatura e a velocidade do vento em Seoul, posso formular a resposta final.

Final Answer: O clima em Seoul é de 15.0°C com uma velocidade do vento de 5.9 km/h.

> Finished chain.

🌤️ Resultado: O clima em Seoul é de 15.0°C com uma velocidade do vento de 5.9 km/h.
